In [32]:
# save cinema locations in Prague

import urllib2
import json
import api_key
import pandas as pd
import colorsys
import time

requestURL = 'https://maps.googleapis.com/maps/api/place/textsearch/json?query=cinemas%20in%20Prague&key=' + api_key.API_KEY

def parse_next(json_data, names, lats, lngs):
    parsed = 0
    for result in json_data['results']:
        if (('name' in result) and ('geometry' in result) and ('location' in result['geometry'])
            and ('lat' in result['geometry']['location']) and ('lng' in result['geometry']['location'])):
            names.append(result['name'])
            lats.append(result['geometry']['location']['lat'])
            lngs.append(result['geometry']['location']['lng'])
            parsed += 1
    return parsed

names = []
lats = []
lngs = []
parsed = 0
to_parse = 200
next_page_token = None

places = pd.DataFrame()

while parsed < to_parse:
    new_address = requestURL
    if next_page_token is not None:
        new_address = requestURL + '&pagetoken=' + str(next_page_token)

    #print(new_address)
    html = urllib2.urlopen(new_address).read().decode('utf-8')
    data = json.loads(html)
    parsed += parse_next(data, names, lats, lngs)

    if 'next_page_token' not in data:
        break
    
    next_page_token = data['next_page_token']
    time.sleep(5)
    
places['name'] = names
places['lat'] = lats
places['lng'] = lngs
places.to_csv("cinema_places.csv", encoding='utf-8')

In [42]:
# parse distances by car

import urllib2
import json
import api_key
import pandas as pd
import colorsys
import time

requestURL = 'https://maps.googleapis.com/maps/api/distancematrix/json?'
key = api_key.API_KEY

my_location = 'Pelc+Tyrolka'
dest_location = ''

def parse_next(json_data, name, distances_names, distances_time, distances_km):
    if 'rows' not in data:
        return

    element = data['rows']
    if len(element) == 0:
        return

    element = element[0]
    if 'elements' not in element:
        return

    element = element['elements']
    if len(element) == 0:
        return

    element = element[0]
    if 'distance' in element:
        distances_names.append(name)
        distances_time.append(element['duration']['text'])
        distances_km.append(element['distance']['text'])

        
distances_names = []
distances_time = []
distances_km = []

places = pd.DataFrame()

i = 0
for lat,lng in zip(lats, lngs):
    address = str(lat) + ',' + str(lng)
    new_address = requestURL + 'origins=' + my_location + '&destinations=' + address.replace(' ', '+') + '&key=' + key

    print(new_address)
    html = urllib2.urlopen(new_address).read().decode('utf-8')
    data = json.loads(html)
    parse_next(data, names[i], distances_names, distances_time, distances_km)
    
    i += 1
    time.sleep(2)
    
places['name'] = distances_names
places['distances_time'] = distances_time
places['distances_km'] = distances_km
places.to_csv("cinema_distances_car.csv", encoding='utf-8')

https://maps.googleapis.com/maps/api/distancematrix/json?origins=Pelc+Tyrolka&destinations=50.0863837,14.4274028&key=%20AIzaSyDAJuwHG8GhkG9ufkxecJhFF3-twhZ2aAw
https://maps.googleapis.com/maps/api/distancematrix/json?origins=Pelc+Tyrolka&destinations=50.090172,14.4719146&key=%20AIzaSyDAJuwHG8GhkG9ufkxecJhFF3-twhZ2aAw
https://maps.googleapis.com/maps/api/distancematrix/json?origins=Pelc+Tyrolka&destinations=50.0999962,14.430005&key=%20AIzaSyDAJuwHG8GhkG9ufkxecJhFF3-twhZ2aAw
https://maps.googleapis.com/maps/api/distancematrix/json?origins=Pelc+Tyrolka&destinations=50.0818326,14.4252589&key=%20AIzaSyDAJuwHG8GhkG9ufkxecJhFF3-twhZ2aAw
https://maps.googleapis.com/maps/api/distancematrix/json?origins=Pelc+Tyrolka&destinations=50.0788488,14.4611085&key=%20AIzaSyDAJuwHG8GhkG9ufkxecJhFF3-twhZ2aAw
https://maps.googleapis.com/maps/api/distancematrix/json?origins=Pelc+Tyrolka&destinations=50.053432,14.517467&key=%20AIzaSyDAJuwHG8GhkG9ufkxecJhFF3-twhZ2aAw
https://maps.googleapis.com/maps/api/distanc

In [45]:
# parse distances by transit (bus, train)

import urllib2
import json
import api_key
import pandas as pd
import colorsys
import time

requestURL = 'https://maps.googleapis.com/maps/api/distancematrix/json?mode=transit&'
key = api_key.API_KEY

my_location = 'Pelc+Tyrolka'
dest_location = ''

def parse_next(json_data, name, distances_names, distances_time, distances_km):
    if 'rows' not in data:
        return

    element = data['rows']
    if len(element) == 0:
        return

    element = element[0]
    if 'elements' not in element:
        return

    element = element['elements']
    if len(element) == 0:
        return

    element = element[0]
    if 'distance' in element:
        distances_names.append(name)
        distances_time.append(element['duration']['text'])
        distances_km.append(element['distance']['text'])

        
distances_names = []
distances_time = []
distances_km = []

places_transit = pd.DataFrame()

i = 0
for lat,lng in zip(lats, lngs):
    address = str(lat) + ',' + str(lng)
    new_address = requestURL + 'origins=' + my_location + '&destinations=' + address.replace(' ', '+') + '&key=' + key

    print(new_address)
    html = urllib2.urlopen(new_address).read().decode('utf-8')
    data = json.loads(html)
    parse_next(data, names[i], distances_names, distances_time, distances_km)
    
    i += 1
    time.sleep(2)
    
places_transit['name'] = distances_names
places_transit['distances_time'] = distances_time
places_transit['distances_km'] = distances_km
places_transit.to_csv("cinema_distances_transit.csv", encoding='utf-8')

https://maps.googleapis.com/maps/api/distancematrix/json?mode=transit&origins=Pelc+Tyrolka&destinations=50.0863837,14.4274028&key=%20AIzaSyDAJuwHG8GhkG9ufkxecJhFF3-twhZ2aAw
https://maps.googleapis.com/maps/api/distancematrix/json?mode=transit&origins=Pelc+Tyrolka&destinations=50.090172,14.4719146&key=%20AIzaSyDAJuwHG8GhkG9ufkxecJhFF3-twhZ2aAw
https://maps.googleapis.com/maps/api/distancematrix/json?mode=transit&origins=Pelc+Tyrolka&destinations=50.0999962,14.430005&key=%20AIzaSyDAJuwHG8GhkG9ufkxecJhFF3-twhZ2aAw
https://maps.googleapis.com/maps/api/distancematrix/json?mode=transit&origins=Pelc+Tyrolka&destinations=50.0818326,14.4252589&key=%20AIzaSyDAJuwHG8GhkG9ufkxecJhFF3-twhZ2aAw
https://maps.googleapis.com/maps/api/distancematrix/json?mode=transit&origins=Pelc+Tyrolka&destinations=50.0788488,14.4611085&key=%20AIzaSyDAJuwHG8GhkG9ufkxecJhFF3-twhZ2aAw
https://maps.googleapis.com/maps/api/distancematrix/json?mode=transit&origins=Pelc+Tyrolka&destinations=50.053432,14.517467&key=%20AIzaSy

In [47]:
print(places)

                                     name distances_time distances_km
0                             Cinema City        14 mins       6.1 km
1                               Kino Aero        13 mins       8.1 km
2                                 BIO OKO         7 mins       2.9 km
3                           Kino Světozor        11 mins       5.7 km
4                             Cinema City        14 mins       6.6 km
5         Premiere Cinemas Praha Hostivař        23 mins      23.3 km
6                            Kino Lucerna        13 mins       6.3 km
7                              Kino Atlas        11 mins       4.5 km
8                      Komorní kino Evald        14 mins       7.0 km
9                             Cinema City        15 mins      11.0 km
10                                    Mat        13 mins       6.5 km
11                     CINEMA CITY Zličín        21 mins      17.2 km
12  Village Cinemas Czech Republic s.r.o.        10 mins       8.8 km
13                  

In [48]:
print(places_transit)

                                     name distances_time distances_km
0                             Cinema City        32 mins       5.0 km
1                               Kino Aero        39 mins       6.6 km
2                                 BIO OKO        19 mins       3.2 km
3                           Kino Světozor        32 mins       5.9 km
4                             Cinema City        35 mins       7.9 km
5         Premiere Cinemas Praha Hostivař        56 mins      17.1 km
6                            Kino Lucerna        33 mins       5.9 km
7                              Kino Atlas        23 mins       4.4 km
8                      Komorní kino Evald        32 mins       6.3 km
9                             Cinema City        40 mins       7.5 km
10                                    Mat        33 mins       7.1 km
11                     CINEMA CITY Zličín  1 hour 2 mins      19.5 km
12  Village Cinemas Czech Republic s.r.o.        43 mins       8.1 km
13                  